*Extracting features of 2018 movies from Wikipedia*

In [9]:
import pandas as pd
import numpy as np
import requests
import json
from tmdbv3api import TMDb
from tmdbv3api import Movie
# The tmdbv3api library in Python is an unofficial wrapper for The Movie Database (TMDb) API version 3. 
# It allows developers to easily interact with the TMDb API to retrieve information about movies, TV shows, and actors.

import warnings
warnings.filterwarnings('ignore')

In [11]:
link="https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
data_1=pd.read_html(link,header=0)[2]
data_2=pd.read_html(link,header=0)[3]
data_3=pd.read_html(link,header=0)[4]
data_4=pd.read_html(link,header=0)[5]


In [12]:
data_1.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.
0,J A N U A R Y,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,J A N U A R Y,5,The Strange Ones,Vertical Entertainment,Christopher Radcliff (director/screenplay); La...,[3]
2,J A N U A R Y,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[4]
3,J A N U A R Y,12,Proud Mary,Screen Gems,"Babak Najafi (director); John S. Newman, Chris...",[5]
4,J A N U A R Y,12,Acts of Violence,Lionsgate Premiere,Brett Donowho (director); Nicolas Aaron Mezzan...,[6]


In [13]:
data = pd.concat([data_1, data_2, data_3, data_4], ignore_index=True)
data

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,J A N U A R Y,5,The Strange Ones,Vertical Entertainment,Christopher Radcliff (director/screenplay); La...,[3],NaN
2,J A N U A R Y,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[4],NaN
3,J A N U A R Y,12,Proud Mary,Screen Gems,"Babak Najafi (director); John S. Newman, Chris...",[5],NaN
4,J A N U A R Y,12,Acts of Violence,Lionsgate Premiere,Brett Donowho (director); Nicolas Aaron Mezzan...,[6],NaN
...,...,...,...,...,...,...,...
236,D E C E M B E R,21,Second Act,STX Entertainment,Peter Segal (director); Elaine Goldsmith-Thoma...,NaN,[229]
237,D E C E M B E R,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions /...,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[140]
238,D E C E M B E R,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[116]
239,D E C E M B E R,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[201]


*As it is observed in the above table, we do not have the genre column so we will get the genre data from IMDb. We will need to create an API key in order to access the API key.*

In [ ]:
# tmdb=TMDb()
# tmdb.api_key='dc6d032ccd5a234bc6937dd5e933075a'

In [15]:
# Set your TMDb API key
tmdb = TMDb()
tmdb.api_key = 'dc6d032ccd5a234bc6937dd5e933075a'

# Initialize the Movie class
tmdb_movie = Movie()

def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)

    if result:
        movie_id = result[0].id

        # Make sure to handle potential errors in the request
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
        if response.status_code == 200:
            data_json = response.json()

            # Check if 'genres' key is present in the response JSON
            if 'genres' in data_json and data_json['genres']:
                genre_str = " "
                for i in range(len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
            else:
                return np.NaN
        else:
            print(f"Error in request for {x}: {response.status_code}")
            return np.NaN
    else:
        print(f"No results found for {x}")
        return np.NaN

# Apply the get_genre function to the 'Title' column and create a new 'genres' column
data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))

data


,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.,genres
0,J A N U A R Y,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN,Horror Mystery Thriller
1,J A N U A R Y,5,The Strange Ones,Vertical Entertainment,Christopher Radcliff (director/screenplay); La...,[3],NaN,Drama Mystery
2,J A N U A R Y,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[4],NaN,Action Thriller Mystery
3,J A N U A R Y,12,Proud Mary,Screen Gems,"Babak Najafi (director); John S. Newman, Chris...",[5],NaN,Thriller Action Crime
4,J A N U A R Y,12,Acts of Violence,Lionsgate Premiere,Brett Donowho (director); Nicolas Aaron Mezzan...,[6],NaN,Action Crime Thriller
...,...,...,...,...,...,...,...,...
236,D E C E M B E R,21,Second Act,STX Entertainment,Peter Segal (director); Elaine Goldsmith-Thoma...,NaN,[229],Romance Comedy
237,D E C E M B E R,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions /...,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[140],Mystery Adventure Comedy Crime
238,D E C E M B E R,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[116],Comedy Drama
239,D E C E M B E R,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[201],Drama History


In [43]:
data_2018_backup=data.copy(deep=True)

In [44]:
data_2018_backup.to_csv("data_2018_backup.csv")

In [46]:
data=pd.read_csv("E:\\Data Science\\Project\\Netflix Recommendation System\\Code\\data_2018_backup.csv")

In [47]:
data_2018=data[['Title','Cast and crew','genres']]
data_2018

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Christopher Radcliff (director/screenplay); La...,Drama Mystery
2,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery
3,Proud Mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime
4,Acts of Violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Crime Thriller
...,...,...,...
236,Second Act,Peter Segal (director); Elaine Goldsmith-Thoma...,Romance Comedy
237,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime
238,Vice,Adam McKay (director/screenplay); Christian Ba...,Comedy Drama
239,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History


*Extracting Directors*

In [48]:
def get_directors(x):
    if " (director)" in x:
        if " (director/screenplay)" in x:
            return x.split(" (director/screenplay)")[0]
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (director);" in x:
        return x.split(" (director); ")[0]
    elif " (director/\u200bscreenplay);" in x:
        return x.split(" (director/\u200bscreenplay);")[0]
    elif " (directors/screenplay);" in x:
        return x.split(" (directors/screenplay)")[0]
    elif " (directors/screenplay)" in x:
        return x.split(" (directors/screenplay)")[0]
    else:
        return x.split(" (director/screenplay)")[0]
    

In [49]:
data_2018['director_name']=data_2018['Cast and crew'].map(lambda x : get_directors(str(x)))

In [50]:
data_2018

,Title,Cast and crew,genres,director_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel
1,The Strange Ones,Christopher Radcliff (director/screenplay); La...,Drama Mystery,Christopher Radcliff
2,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra
3,Proud Mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime,Babak Najafi
4,Acts of Violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Crime Thriller,Brett Donowho
...,...,...,...,...
236,Second Act,Peter Segal (director); Elaine Goldsmith-Thoma...,Romance Comedy,Peter Segal
237,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen
238,Vice,Adam McKay (director/screenplay); Christian Ba...,Comedy Drama,Adam McKay
239,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder


In [51]:
data_2018['director_name'][92]=data_2018['director_name'][92].split(" (director); ")[0]


In [52]:

data_2018[data_2018['director_name'].str.contains('(director)')]

,Title,Cast and crew,genres,director_name


*Actor 1*

In [53]:
def get_actor_1(x):
    return (x.split(',')[-1])

In [54]:
data_2018['actor_1_name']=data_2018['Cast and crew'].map(lambda x : get_actor_1(x))
data_2018

,Title,Cast and crew,genres,director_name,actor_1_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Bruce Davison
1,The Strange Ones,Christopher Radcliff (director/screenplay); La...,Drama Mystery,Christopher Radcliff,Tobias Campbell
2,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Sam Neill
3,Proud Mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime,Babak Najafi,Danny Glover
4,Acts of Violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Crime Thriller,Brett Donowho,Mike Epps
...,...,...,...,...,...
236,Second Act,Peter Segal (director); Elaine Goldsmith-Thoma...,Romance Comedy,Peter Segal,Milo Ventimiglia
237,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Hugh Laurie
238,Vice,Adam McKay (director/screenplay); Christian Ba...,Comedy Drama,Adam McKay,Jesse Plemons
239,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Kathy Bates


*Actor 2*

In [55]:
def get_actor_2(x):
    if len(x.split(','))<2:
        return np.nan
    return (x.split(',')[-2])

In [56]:
data_2018['actor_2_name']=data_2018['Cast and crew'].map(lambda x : get_actor_2(x))
data_2018

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Bruce Davison,Caitlin Gerard
1,The Strange Ones,Christopher Radcliff (director/screenplay); La...,Drama Mystery,Christopher Radcliff,Tobias Campbell,Owen Campbell
2,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Sam Neill,Jonathan Banks
3,Proud Mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime,Babak Najafi,Danny Glover,Billy Brown
4,Acts of Violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Crime Thriller,Brett Donowho,Mike Epps,Sophia Bush
...,...,...,...,...,...,...
236,Second Act,Peter Segal (director); Elaine Goldsmith-Thoma...,Romance Comedy,Peter Segal,Milo Ventimiglia,Treat Williams
237,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Hugh Laurie,Pam Ferris
238,Vice,Adam McKay (director/screenplay); Christian Ba...,Comedy Drama,Adam McKay,Jesse Plemons,Lily Rabe
239,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Kathy Bates,Sam Waterston


Actor 3 

In [57]:
def get_actor_3(x):
    if len(x.split(','))<3:
        return np.nan
    return (x.split(',')[-3])

In [58]:
data_2018['actor_3_name']=data_2018['Cast and crew'].map(lambda x : get_actor_2(x))
data_2018

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Bruce Davison,Caitlin Gerard,Caitlin Gerard
1,The Strange Ones,Christopher Radcliff (director/screenplay); La...,Drama Mystery,Christopher Radcliff,Tobias Campbell,Owen Campbell,Owen Campbell
2,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Sam Neill,Jonathan Banks,Jonathan Banks
3,Proud Mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime,Babak Najafi,Danny Glover,Billy Brown,Billy Brown
4,Acts of Violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Crime Thriller,Brett Donowho,Mike Epps,Sophia Bush,Sophia Bush
...,...,...,...,...,...,...,...
236,Second Act,Peter Segal (director); Elaine Goldsmith-Thoma...,Romance Comedy,Peter Segal,Milo Ventimiglia,Treat Williams,Treat Williams
237,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Hugh Laurie,Pam Ferris,Pam Ferris
238,Vice,Adam McKay (director/screenplay); Christian Ba...,Comedy Drama,Adam McKay,Jesse Plemons,Lily Rabe,Lily Rabe
239,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Kathy Bates,Sam Waterston,Sam Waterston


*Renaming The Column*

In [59]:
data_2018=data_2018.rename(columns={'Title':'movie_title'})

In [60]:
new_data_2018=data_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_data_2018

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Bruce Davison,Caitlin Gerard,Caitlin Gerard,Horror Mystery Thriller,Insidious: The Last Key
1,Christopher Radcliff,Tobias Campbell,Owen Campbell,Owen Campbell,Drama Mystery,The Strange Ones
2,Jaume Collet-Serra,Sam Neill,Jonathan Banks,Jonathan Banks,Action Thriller Mystery,The Commuter
3,Babak Najafi,Danny Glover,Billy Brown,Billy Brown,Thriller Action Crime,Proud Mary
4,Brett Donowho,Mike Epps,Sophia Bush,Sophia Bush,Action Crime Thriller,Acts of Violence
...,...,...,...,...,...,...
236,Peter Segal,Milo Ventimiglia,Treat Williams,Treat Williams,Romance Comedy,Second Act
237,Etan Cohen,Hugh Laurie,Pam Ferris,Pam Ferris,Mystery Adventure Comedy Crime,Holmes & Watson
238,Adam McKay,Jesse Plemons,Lily Rabe,Lily Rabe,Comedy Drama,Vice
239,Mimi Leder,Kathy Bates,Sam Waterston,Sam Waterston,Drama History,On the Basis of Sex


In [61]:
new_data_2018['actor_2_name']=new_data_2018['actor_2_name'].replace(np.nan,'unknown')
new_data_2018['actor_3_name']=new_data_2018['actor_3_name'].replace(np.nan,'unknown')

In [62]:
new_data_2018.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [63]:
new_data_2018['movie_title']=new_data_2018['movie_title'].str.lower()

In [64]:
new_data_2018['comb']=new_data_2018['actor_1_name']+" "+new_data_2018['actor_2_name']+" "+new_data_2018['actor_3_name']+" "+new_data_2018['director_name']+" "+new_data_2018['genres']

In [65]:
new_data_2018.to_csv("data_3.csv",index=False)